In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Simulation
#Misura di tutti i qubit
n=2
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i])
    


#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
print(circuits[-1].parameters)
circuits[-1].draw(output='mpl')

#circuits[1].draw()


job = execute(circ,
            backend=BasicAer.get_backend('qasm_simulator'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range])

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00')
print(list(map(lambda c: 1/1024*c.get('00', 0), counts2)))

Phi range
[0.0, 1.0471975511965976, 2.0943951023931953, 3.141592653589793, 4.1887902047863905, 5.235987755982989]
set()


Counts
[{'00': 1024}, {'01': 757, '00': 267}, {'01': 772, '00': 252}, {'00': 1024}, {'01': 765, '00': 259}, {'01': 735, '00': 289}]


Counts of 00
[1.0, 0.2607421875, 0.24609375, 1.0, 0.2529296875, 0.2822265625]


In [3]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 2
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmqx2'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00')
print(list(map(lambda c: 1/shots*c.get('00', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 1.0471975511965976, 2.0943951023931953, 3.141592653589793, 4.1887902047863905, 5.235987755982989]
Job Status: job has successfully run


Counts
[{'01': 328, '11': 164, '10': 359, '00': 7341}, {'01': 5285, '11': 184, '10': 258, '00': 2465}, {'01': 5571, '11': 168, '10': 250, '00': 2203}, {'01': 332, '11': 140, '10': 310, '00': 7410}, {'01': 5208, '11': 150, '10': 287, '00': 2547}, {'01': 5576, '11': 178, '10': 284, '00': 2154}]


Counts of 00
[0.8961181640625, 0.3009033203125, 0.2689208984375, 0.904541015625, 0.3109130859375, 0.262939453125]
JOB ID: 5d3f1833e454ab0078ec72cc


In [4]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 2
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmqx2')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00')
print(list(map(lambda c: 1/shots*c.get('00', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 1.0471975511965976, 2.0943951023931953, 3.141592653589793, 4.1887902047863905, 5.235987755982989]
Job Status: job has successfully run


Counts
[{'01': 336, '11': 202, '10': 357, '00': 7297}, {'01': 5217, '11': 229, '10': 338, '00': 2408}, {'01': 5512, '11': 197, '10': 279, '00': 2204}, {'01': 370, '11': 150, '10': 341, '00': 7331}, {'01': 5137, '11': 191, '10': 323, '00': 2541}, {'01': 5454, '11': 228, '10': 342, '00': 2168}]


Counts of 00
[0.8907470703125, 0.2939453125, 0.26904296875, 0.8948974609375, 0.3101806640625, 0.2646484375]
JOB ID: 5d3f18c6d8bac900728f3b96


In [ ]:
#from qiskit.providers.ibmq import least_busy
#backend = least_busy(provider.backends(filters=lambda x: not x.configuration().simulator))
#backend.name()

In [5]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 2
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 2, q[1]: 1}

backend = IBMQ.get_backend('ibmqx2')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range], shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 00')
print(list(map(lambda c: 1/shots*c.get('00', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 1.0471975511965976, 2.0943951023931953, 3.141592653589793, 4.1887902047863905, 5.235987755982989]
Job Status: job has successfully run


Counts
[{'01': 341, '11': 167, '10': 313, '00': 7371}, {'01': 5297, '11': 161, '10': 325, '00': 2409}, {'01': 5591, '11': 169, '10': 279, '00': 2153}, {'01': 379, '11': 139, '10': 280, '00': 7394}, {'01': 5207, '11': 193, '10': 342, '00': 2450}, {'01': 5564, '11': 205, '10': 288, '00': 2135}]


Counts of 00
[0.8997802734375, 0.2940673828125, 0.2628173828125, 0.902587890625, 0.299072265625, 0.2606201171875]
JOB ID: 5d3f19538968ca0073ef7443
